Problem 2 

 definition EER:
The effective exchange rate is an index that describes the strength of a currency relative to a basket of other currencies. Suppose a country has N trading partners and denote $Trade_{i}$ and $E_{i}$  as the trade and exchange rate with country $i$  respectively. Then the effective exchange rate is calculated as:

$${\displaystyle E_{effective}=E_{1}{\frac {Trade_{1}}{Trade}}+...+E_{N}{\frac {Trade_{N}}{Trade}}}$$

 definition REER:
A real effective exchange rate (REER) adjusts EER by the appropriate foreign price level and deflates by the home country price level

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm

C:\Users\phbs\Anaconda3\lib\site-packages\statsmodels\compat\pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [14]:
erf = pd.read_csv('https://github.com/QuantEcon/ShenzhenWinterCamp/raw/master/Jiao%20Shi/ER_predict.csv')
erf.head()

,Date,CNYUSD,RMB_REER,CHN PMI,USA PMI,CHN M1,USA M1,CHN M2,USA M2,CHN CPI inflation,USA CPI inflation,CHN_IBR,US_FFR
0,2005-07,8.1062,85.01,51.1,52.8,"97,700.00","1,365.00","277,000.00","6,498.40",1.8,3.2,1.1395,3.36
1,2005-08,8.0998,85.05,52.6,52.4,"99,400.00","1,376.50","281,300.00","6,522.50",1.3,3.6,1.1774,3.30
2,2005-09,8.0920,85.29,55.1,56.8,"101,000.00","1,363.10","287,000.00","6,555.00",0.9,4.7,1.1675,3.57
3,2005-10,8.0845,86.39,54.1,57.2,"101,800.00","1,365.10","287,600.00","6,582.30",1.2,4.3,1.2442,3.73
4,2005-11,8.0804,87.75,54.1,56.7,"104,100.00","1,373.20","292,400.00","6,623.00",1.3,3.5,1.2049,3.99


The Taylor-rule model
$$ E_{t}(s_{t+1}-s_{t})=\alpha+\beta_1(\pi_{t}-\pi_{t}^{*})+\beta_{2}(\tilde{y}_{t}-\tilde{y}_{t}^{*}) $$

In [15]:
erf['Δπ'] = erf['CHN CPI inflation']-erf['USA CPI inflation']
erf['Δy'] = (erf['CHN PMI'] - np.mean(erf['CHN PMI'])) - (erf['USA PMI'] - np.mean(erf['USA PMI']))
erf['ΔREER_Notconfig'] = erf['RMB_REER'].diff(periods = 1)
erf['Constant'] = 1
erf['ΔREER'] = erf['ΔREER_Notconfig'].shift(-1)

In [17]:
result = sm.OLS(erf['ΔREER'][:len(erf)-1],erf[['Constant','Δπ','Δy']][:len(erf)-1]).fit()
print(result.summary())

                            OLS Regression Results                            
Dep. Variable:                  ΔREER   R-squared:                       0.010
Model:                            OLS   Adj. R-squared:                 -0.003
Method:                 Least Squares   F-statistic:                    0.7599
Date:                Fri, 02 Feb 2018   Prob (F-statistic):              0.470
Time:                        15:58:41   Log-Likelihood:                -262.78
No. Observations:                 149   AIC:                             531.6
Df Residuals:                     146   BIC:                             540.6
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Constant       0.2457      0.126      1.948      0.0

In [27]:
##Compute the RMSE of raw Taylor rule model
np.sqrt( np.mean((result.predict()-erf['ΔREER'][:len(erf)-1])**2) )

1.4115433851784678

**the sign of coefficient is positive, it is consistent with our intuition but not statistically significant**

In [23]:
size_sample = 149
size_subsample = 60
fcst_tr = np.zeros(size_sample - size_subsample)
fcst_error_tr = np.zeros(size_sample - size_subsample)
for i in range (size_sample - size_subsample):
    # Create a column to store the output gap difference and update it in each loop
    y_chn = erf['CHN PMI'] - np.mean(erf.loc[i:size_subsample-1+i, 'CHN PMI'])
    y_us = erf['USA PMI'] - np.mean(erf.loc[i:size_subsample-1+i, 'USA PMI'])
    erf['y_diff'] = y_chn - y_us
    
    reg_trsub = sm.OLS(endog=erf.loc[i:size_subsample-1+i,'ΔREER'], exog=erf.loc[erf.index[i:size_subsample+i],['Constant', 'Δπ','y_diff']], missing='drop')
    results_sub = reg_trsub.fit()
    
    
    fcst_tr[i] = results_sub.predict(exog = [erf.loc[size_subsample+i,['Constant', 'Δπ','y_diff']]])
    error = results_sub.predict(exog = [erf.loc[size_subsample+i,['Constant', 'Δπ','y_diff']]]) - erf.loc[size_subsample+i, 'ΔREER']
    fcst_error_tr[i] = error
    

# Compute RMSE for adjusted Taylor rule model
rmse_tr = np.sqrt(np.mean(fcst_error_tr**2))
rmse_tr

1.4288842291860202

**Comparing with this morning's Taylor Rule Model, we find that RMSE becomes larger.**